In [1]:
from time import sleep
import numpy as np
import matplotlib.pyplot as plt #py -m install mataplotlib
import pandas as pd  #py -m install pandas
import io
import inspect
import datetime
from pandasql import sqldf, load_births
import requests  #pip3 -m install requests. Make sure to link pip3 to environment variable path
import os
import sqlite3
import time
import time
import glob
import os.path
from pathlib import Path

In [2]:
dbname = 'TWStock.db'

In [3]:
db = sqlite3.connect("TWStock.db")

In [4]:
def crawler(date_time):
    url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+ date_time +'&type=ALLBUT0999'
    time.sleep(3)
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
        if text == '"證券代號","證券名稱","成交股數","成交筆數","成交金額","開盤價","最高價","最低價","收盤價","漲跌(+/-)","漲跌價差","最後揭示買價","最後揭示買量","最後揭示賣價","最後揭示賣量","本益比",':
            initial_point = i
            break
    test_df = pd.read_csv(io.StringIO(''.join([text[:-1] + '\n'for text in use_text[initial_point:]])))
    test_df['證券代號'] = test_df['證券代號'].apply(lambda x:x.replace('"',''))
    test_df['證券代號'] = test_df['證券代號'].apply(lambda x: x.replace('=',''))
    return test_df

In [5]:

def trans_date(date_time):
    return ''.join(str(date_time).split('-'))[0:8]


In [6]:
def parse_date(start_date, n):
    df_dict = {}
    time.sleep(3)
    now_date = start_date
    for i in range(n):
        now_date = now_date - datetime.timedelta(days=1)
        try:
            df = crawler(trans_date(now_date))
            print('successful at'+' '+ trans_date(now_date))
            df_dict.update({trans_date(now_date):df})
            time.sleep(3)
        except:
            print('fail at' + ' ' + trans_date(now_date))
            time.sleep(3)
    return df_dict


In [7]:
%%time

result_dict = parse_date(datetime.datetime.now(),2)

fail at 20210116
successful at 20210115
Wall time: 18.1 s


In [82]:
result_dict.keys()

dict_keys(['20210115', '20210114'])

In [7]:
for key in result_dict.keys():
    result_dict[key].to_csv(str(key)+'.csv')

In [8]:
result_dict['20210105']['成交股數']

KeyError: '20210105'

In [8]:
ALL_csv_file = glob.glob('*.csv')

In [9]:
%%time
for file_name in ALL_csv_file:
    pd.read_csv(file_name).iloc[:,1:].to_sql(file_name.replace('.csv',''),db,if_exists='replace')

Wall time: 78.8 ms


In [10]:
cf = ALL_csv_file

In [11]:
date_list = [file_name.replace('.csv','') for file_name in ALL_csv_file]

In [12]:
total_df = pd.DataFrame()
for date in date_list:
    df = pd.read_sql(con=db,sql='SELECT * FROM' + ' "'+ date +'"')
    df['Date'] = date
    total_df = total_df.append(df)

In [13]:
total_df

,index,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Date
0,0,0050,元大台灣50,"12,390,589","11,405","1,660,294,742",135.00,135.05,133.35,134.05,-,1.25,134.05,228,134.10,70,0.00,20210114
1,1,0051,元大中型100,"45,897",64,"2,096,825",45.62,45.80,45.55,45.75,+,0.13,45.70,2,45.74,35,0.00,20210114
2,2,0052,富邦科技,"1,751,591","1,584","215,555,508",122.95,123.90,122.20,122.80,-,2.20,122.75,19,122.80,22,0.00,20210114
3,3,0053,元大電子,"40,484",48,"2,573,066",63.65,64.00,63.30,63.85,-,0.30,63.80,1,63.85,10,0.00,20210114
4,4,0054,元大台商50,"63,265",35,"1,943,458",30.27,30.90,30.27,30.85,+,0.56,30.79,52,30.90,4,0.00,20210114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,1116,9944,新麗,"404,848",238,"8,778,124",22.10,22.10,21.30,21.60,-,0.55,21.60,8,21.65,11,63.53,20210115
1117,1117,9945,潤泰新,"4,709,150","2,546","190,263,316",40.70,40.80,40.10,40.25,-,0.45,40.25,20,40.30,18,8.71,20210115
1118,1118,9946,三發地產,"659,927",294,"9,673,121",14.80,14.90,14.50,14.90,-,0.05,14.65,7,14.90,21,8.51,20210115
1119,1119,9955,佳龍,"531,202",346,"10,098,243",19.50,19.50,18.80,18.90,-,0.50,18.90,17,19.00,6,0.00,20210115


In [14]:
dbname_2 = 'TWStock_2'
db2 = sqlite3.connect(dbname_2)


In [15]:
total_dict = dict(tuple(total_df.groupby('證券代號')))
for key in total_dict.keys():
    df = total_dict[key].iloc[:,2:]
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by=['Date'])
    df.to_sql(key,db2,if_exists='replace')

In [16]:
total_df['成交股數']=total_df['成交股數'].str.replace(',','')
total_df['成交筆數']=total_df['成交筆數'].str.replace(',','')

In [17]:
total_df['成交股數'] = total_df['成交股數'].astype('str') 
total_df['成交筆數'] = total_df['成交筆數'].astype('str') 

In [18]:
total_df

,index,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Date
0,0,0050,元大台灣50,12390589,11405,"1,660,294,742",135.00,135.05,133.35,134.05,-,1.25,134.05,228,134.10,70,0.00,20210114
1,1,0051,元大中型100,45897,64,"2,096,825",45.62,45.80,45.55,45.75,+,0.13,45.70,2,45.74,35,0.00,20210114
2,2,0052,富邦科技,1751591,1584,"215,555,508",122.95,123.90,122.20,122.80,-,2.20,122.75,19,122.80,22,0.00,20210114
3,3,0053,元大電子,40484,48,"2,573,066",63.65,64.00,63.30,63.85,-,0.30,63.80,1,63.85,10,0.00,20210114
4,4,0054,元大台商50,63265,35,"1,943,458",30.27,30.90,30.27,30.85,+,0.56,30.79,52,30.90,4,0.00,20210114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,1116,9944,新麗,404848,238,"8,778,124",22.10,22.10,21.30,21.60,-,0.55,21.60,8,21.65,11,63.53,20210115
1117,1117,9945,潤泰新,4709150,2546,"190,263,316",40.70,40.80,40.10,40.25,-,0.45,40.25,20,40.30,18,8.71,20210115
1118,1118,9946,三發地產,659927,294,"9,673,121",14.80,14.90,14.50,14.90,-,0.05,14.65,7,14.90,21,8.51,20210115
1119,1119,9955,佳龍,531202,346,"10,098,243",19.50,19.50,18.80,18.90,-,0.50,18.90,17,19.00,6,0.00,20210115


In [19]:
td2 = total_df[['證券代號','證券名稱','成交股數','成交筆數','Date']]

In [20]:
td2

,證券代號,證券名稱,成交股數,成交筆數,Date
0,0050,元大台灣50,12390589,11405,20210114
1,0051,元大中型100,45897,64,20210114
2,0052,富邦科技,1751591,1584,20210114
3,0053,元大電子,40484,48,20210114
4,0054,元大台商50,63265,35,20210114
...,...,...,...,...,...
1116,9944,新麗,404848,238,20210115
1117,9945,潤泰新,4709150,2546,20210115
1118,9946,三發地產,659927,294,20210115
1119,9955,佳龍,531202,346,20210115


In [21]:
td2['成交股數'] = pd.to_numeric(td2["成交股數"])
td2['成交筆數'] = pd.to_numeric(td2["成交筆數"])

<ipython-input-21-2f4c17fe87ce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  td2['成交股數'] = pd.to_numeric(td2["成交股數"])
<ipython-input-21-2f4c17fe87ce>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  td2['成交筆數'] = pd.to_numeric(td2["成交筆數"])


In [22]:
%%time
container=[]
for number, _df in td2.groupby(['證券代號']):
    _df.loc['{number[0]}, {number[1]} subtotal'] = _df[['成交股數','成交筆數']].sum()

Wall time: 1min 51s


In [121]:
df_summary

,證券代號,證券名稱,成交股數,成交筆數,Date
0,0050,元大台灣50,12390589.0,11405.0,20210114
0,0050,元大台灣50,21933080.0,13533.0,20210115
,NaN,NaN,34323669.0,24938.0,NaN
1,0051,元大中型100,45897.0,64.0,20210114
1,0051,元大中型100,78484.0,75.0,20210115
...,...,...,...,...,...
1119,9955,佳龍,531202.0,346.0,20210115
,NaN,NaN,834113.0,562.0,NaN
1119,9958,世紀鋼,3420332.0,2289.0,20210114
1120,9958,世紀鋼,3355539.0,2479.0,20210115


In [50]:
df_summary['Date'] = pd.to_datetime(df_summary['Date'])

In [157]:
df_summary.loc[''=="證券代號"] = _df[['成交股數','成交筆數']].sum()

ValueError: Must have equal len keys and value when setting with an iterable

In [73]:
df_summary.loc[1]

,證券代號,證券名稱,成交股數,成交筆數,Date
1,0051,元大中型100,45897.0,64.0,2021-01-14
1,0051,元大中型100,78484.0,75.0,2021-01-15


In [156]:
df_summary.iloc[0,0]

'0050'

In [141]:
%%time
container=[]
for number, _df in td2.groupby(['證券代號']):
    _df.loc['subtotal'] = _df[['成交股數','成交筆數']].sum()
    container.append(_df)
    df_summary = pd.concat(container)

Wall time: 2min 10s


In [163]:

%%time
container=[]
for number, _df in td2.groupby(['證券代號']):
    _df.loc['total_dict'] = _df[['成交股數','成交筆數']].sum()
    container.append(_df)
    df_summary = pd.concat(container)

Wall time: 2min 20s


In [105]:
for x in total_dict:
    td2.loc[]
    

0050
0051
0052
0053
0054
0055
0056
0057
0061
006203
006204
006205
006206
006207
006208
00625K
00631L
00632R
00633L
00634R
00635U
00636
00636K
00637L
00638R
00639
00640L
00641R
00642U
00643
00643K
00645
00646
00647L
00648R
00650L
00651R
00652
00653L
00654R
00655L
00656R
00657
00657K
00660
00661
00662
00663L
00664R
00665L
00666R
00668
00668K
00669R
00670L
00671R
00673R
00674R
00675L
00676R
00677U
00678
00680L
00681R
00682U
00683L
00684R
00685L
00686R
00688L
00689R
00690
00692
00693U
00700
00701
00702
00703
00706L
00707R
00708L
00709
00710B
00711B
00712
00713
00714
00715L
00717
00728
00730
00731
00732
00733
00735
00736
00737
00738U
00739
00742
00743
00752
00753L
00757
00762
00763U
00770
00771
00774B
00774C
00775B
00783
008201
00830
00850
00851
00852L
00861
00865B
00875
00876
00878
00881
01001T
01002T
01003T
01004T
01007T
01009T
01010T
020000
020002
020004
020006
020007
020008
020011
020012
020015
020016
020018
020019
02001L
02001R
020020
020022
1101
1101B
1102
1103
1104
1108
1109
1110
120

In [134]:
td2

,證券代號,證券名稱,成交股數,成交筆數,Date
0,0050,元大台灣50,12390589,11405,20210114
1,0051,元大中型100,45897,64,20210114
2,0052,富邦科技,1751591,1584,20210114
3,0053,元大電子,40484,48,20210114
4,0054,元大台商50,63265,35,20210114
...,...,...,...,...,...
1116,9944,新麗,404848,238,20210115
1117,9945,潤泰新,4709150,2546,20210115
1118,9946,三發地產,659927,294,20210115
1119,9955,佳龍,531202,346,20210115


In [145]:
td2.iloc[0,0]

'0050'

In [162]:
df_summary

,證券代號,證券名稱,成交股數,成交筆數,Date
0,0050,元大台灣50,12390589,11405,20210114
0,0050,元大台灣50,21933080,13533,20210115
1,0051,元大中型100,45897,64,20210114
1,0051,元大中型100,78484,75,20210115
2,0052,富邦科技,1751591,1584,20210114
2,0052,富邦科技,2110386,1633,20210115
3,0053,元大電子,40484,48,20210114
3,0053,元大電子,68598,60,20210115
4,0054,元大台商50,63265,35,20210114
4,0054,元大台商50,70300,21,20210115


In [90]:
df_summary.set_index(['證券代號','Date'])

證券名稱        成交股數     成交筆數
證券代號 Date                                    
0050 2021-01-14   元大台灣50  12390589.0  11405.0
     2021-01-15   元大台灣50  21933080.0  13533.0
NaN  NaT             NaN  34323669.0  24938.0
0051 2021-01-14  元大中型100     45897.0     64.0
     2021-01-15  元大中型100     78484.0     75.0
...                  ...         ...      ...
9955 2021-01-15       佳龍    531202.0    346.0
NaN  NaT             NaN    834113.0    562.0
9958 2021-01-14      世紀鋼   3420332.0   2289.0
     2021-01-15      世紀鋼   3355539.0   2479.0
NaN  NaT             NaN   6775871.0   4768.0

[3362 rows x 3 columns]

In [93]:
df_summary.loc[(2,'Date')]

0   2021-01-14
0   2021-01-15
Name: Date, dtype: datetime64[ns]

In [129]:
df_summary.loc[1]

,證券代號,證券名稱,成交股數,成交筆數,Date
1,0051,元大中型100,45897.0,64.0,20210114
1,0051,元大中型100,78484.0,75.0,20210115


12390589.0

In [155]:
for x in range(0,2000):
    ds = df.loc[x].iloc[0,2]
    print(ds)

12390589.0
45897.0
1751591.0
40484.0
63265.0
151406.0
29597950.0
21022.0
355173.0
8004.0
4128.0
2016978.0
906385.0
325112.0
2390881.0
20000.0
2868088.0
109543964.0
10966200.0
1476100.0
1464320.0
2077391.0
100.0
75122045.0
1147000.0
2337330.0
1002005.0
316000.0
24416657.0
1406507.0
0.0
474073.0
1851695.0
54390.0
424000.0
413000.0
307000.0
97281.0
1344785.0
129000.0
4702150.0
1029000.0
9000.0
100.0
23139.0
8000.0
181619.0
17000.0
4214000.0
352000.0
80000.0
14000.0
100.0
2839000.0
438073.0
1337000.0
461100.0
543000.0
397205.0
7737005.0
692109468.0
285684.0
89350.0
239000.0
109005.0
2398000.0
0.0
29000.0
92000.0
187000.0
102000.0
256667.0
3048718.0
4276616.0
158471.0
2941668.0
10069.0
216640.0
3000.0
588000.0
4703150.0
397370.0
216000.0
117100.0
1136605.0
190205.0
287200.0
68580483.0
776350.0
1174403.0
586220.0
1029783.0
101000.0
257234.0
127000.0
172000.0
323105.0
6659756.0
899612.0
71050.0
493206.0
3487559.0
568000.0
622765.0
172730.0
191050.0
75068.0
51555.0
2000.0
0.0
2000.0
742120.0
3

446728.0
887991.0
1262320.0
1136147.0
18100.0
140388.0
895287.0
703242.0
4090911.0
503229.0
984635.0
55220.0
32000.0
16001.0
49618.0
610039.0
1119804.0
234554.0
42000.0
543223.0
54100.0
479897.0
2987708.0
69096.0
418150.0
23000.0
797830.0
5095369.0
4232696.0
7526273.0
8769263.0
2062369.0
3069299.0
11246217.0
5301464.0
168622.0
1341328.0
229611.0
1233223.0
562598.0
3572604.0
3958898.0
3446652.0
252811.0
815620.0
13405236.0
2801743.0
125031.0
279278.0
3206574.0
4412745.0
188287.0
409037.0
6890827.0
2245950.0
111257.0
592527.0
211016.0
260139.0
102882.0
267227.0
40000.0
1237637.0
45000.0
86369.0
321406.0
81741.0
151031.0
877950.0
29313.0
18000.0
103693.0
196600.0
30700.0
25000.0
142001.0
6000.0
234577.0
2003969.0
116469.0
1435877.0
5779056.0
16000.0
9042161.0
1120170.0
401000.0
323000.0
418299.0
408000.0
1112000.0
4071052.0
448623.0
284000.0
266000.0
834635.0
1409832.0
8747558.0
209088.0
63650.0
4534677.0
143438.0
1743644.0
424505.0
447992.0
1279326.0
386952.0
140637.0
2626974.0
1532451.0

IndexingError: Too many indexers

In [144]:
df_summary.tail(20)

,證券代號,證券名稱,成交股數,成交筆數,Date
1114,9942,茂順,168413.0,141.0,20210115
subtotal,NaN,NaN,297592.0,247.0,NaN
1114,9943,好樂迪,155460.0,126.0,20210114
1115,9943,好樂迪,154507.0,142.0,20210115
subtotal,NaN,NaN,309967.0,268.0,NaN
1115,9944,新麗,379769.0,218.0,20210114
1116,9944,新麗,404848.0,238.0,20210115
subtotal,NaN,NaN,784617.0,456.0,NaN
1116,9945,潤泰新,3452519.0,1557.0,20210114
1117,9945,潤泰新,4709150.0,2546.0,20210115


In [149]:
df_summary.loc[0,'證券代號']

0    0050
0    0050
Name: 證券代號, dtype: object

In [150]:
showdis = df_summary[['證券代號','成交股數','Date']]

In [152]:
showdis.columns = ['a', 'b', 'c']

In [154]:
print(showdis)

               a             b         c
0           0050  1.239059e+07  20210114
0           0050  2.193308e+07  20210115
subtotal     NaN  3.432367e+07       NaN
1           0051  4.589700e+04  20210114
1           0051  7.848400e+04  20210115
subtotal     NaN  1.243810e+05       NaN
2           0052  1.751591e+06  20210114
2           0052  2.110386e+06  20210115
subtotal     NaN  3.861977e+06       NaN
3           0053  4.048400e+04  20210114
3           0053  6.859800e+04  20210115
subtotal     NaN  1.090820e+05       NaN
4           0054  6.326500e+04  20210114
4           0054  7.030000e+04  20210115
subtotal     NaN  1.335650e+05       NaN
5           0055  1.514060e+05  20210114
5           0055  1.165870e+05  20210115
subtotal     NaN  2.679930e+05       NaN
6           0056  2.959795e+07  20210114
6           0056  2.132365e+07  20210115
subtotal     NaN  5.092160e+07       NaN
7           0057  2.102200e+04  20210114
7           0057  2.993000e+04  20210115
subtotal     NaN